In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import math
import scipy
import random
from catboost import CatBoostRegressor
%matplotlib inline

In [2]:
# Load datasets
df_train = pd.read_csv('tcd-ml-1920-group-income-train.csv')
df_test = pd.read_csv('tcd-ml-1920-group-income-test.csv')
df_sub = pd.read_csv('tcd-ml-1920-group-income-submission.csv')

# Define shorter names
new_columns = {
    'Year of Record': 'Year',
    'Housing Situation': 'House',
    'Crime Level in the City of Employement': 'Crime',
    'Work Experience in Current Job [years]': 'Experience',
    'Satisfation with employer': 'Satisfaction',
    'University Degree':'Degree',
    'Wears Glasses': 'Glasses',
    'Hair Color': 'Hair',
    'Body Height [cm]':'Height',
    'Yearly Income in addition to Salary (e.g. Rental Income)': 'Other Income',
    'Total Yearly Income [EUR]': 'Income'
          }

# Rename columns
df_train.rename(columns = new_columns, inplace = True)
df_test.rename(columns = new_columns, inplace = True)

# Remove trailing 'EUR' from entries
df_test['Other Income'] = df_test['Other Income'].str[:-4]
df_train['Other Income'] = df_train['Other Income'].str[:-4]

# Convert from string to int
df_test['Other Income'] = pd.to_numeric( df_test['Other Income'], errors='coerce').astype(np.int32)
df_train['Other Income'] = pd.to_numeric( df_train['Other Income'], errors='coerce').astype(np.int32)

# # Only a handful of '#NUM!' so drop these rows
# df_test = (df_test[df_test['Experience'] != '#NUM!'])
# df_train = (df_train[df_train['Experience'] != '#NUM!'])

# # Only 4013 missing years so drop
# df_test = (df_test[df_test['Year'].notnull()])
# df_train = (df_train[df_train['Year'].notnull()])

# # Only 2805 missing Professions so drop
# df_test = df_test[df_test['Profession'].notnull()]
# df_train = df_train[df_train['Profession'].notnull()]

# # There's 1 missing Country in test set so drop row
# df_test = df_test[df_test['Country'].notnull()]

# Convert strings to floats
df_test['Experience'] = pd.to_numeric(df_test['Experience'], errors='coerce')
df_train['Experience'] = pd.to_numeric(df_train['Experience'], errors='coerce')

#  Fix missing values
exp_test_median = df_test['Experience'].median()
exp_train_median = df_train['Experience'].median()
df_test['Experience'].fillna(exp_test_median, inplace=True)
df_train['Experience'].fillna(exp_train_median, inplace=True)

year_test_median = df_test['Year'].median()
year_train_median = df_train['Year'].median()
df_test['Year'].fillna(year_test_median, inplace=True)
df_train['Year'].fillna(year_train_median, inplace=True)

df_test['Profession'].fillna('Unknown', inplace=True)
df_train['Profession'].fillna('Unknown', inplace=True)

df_test['Country'].fillna('Unknown', inplace=True)
df_train['Country'].fillna('Unknown', inplace=True)

# Convert to ints for space
df_test['Year'] = df_test['Year'].astype(np.int16)
df_train['Year'] = df_train['Year'].astype(np.int16)

df_test['Experience'] = df_test['Experience'].astype(np.int8)
df_train['Experience'] = df_train['Experience'].astype(np.int8)

df_test['Crime'] = df_test['Crime'].astype(np.uint8)
df_train['Crime'] = df_train['Crime'].astype(np.uint8)

df_test['Age'] = df_test['Age'].astype(np.uint8)
df_train['Age'] = df_train['Age'].astype(np.uint8)

df_test['Height'] = df_test['Height'].astype(np.int16)
df_train['Height'] = df_train['Height'].astype(np.int16)

# Drop Instance Column
df_test.drop('Instance', inplace=True, axis=1)
df_train.drop('Instance', inplace=True, axis=1)

# Drop Income column from test set
df_test.drop('Income', inplace=True, axis=1)

# Replace NaNs merge 'f' and 'female'
df_test['Satisfaction'].fillna('Unknown', inplace=True)
df_train['Satisfaction'].fillna('Unknown', inplace=True)

df_test['Gender'].fillna('Unknown', inplace=True)
df_train['Gender'].fillna('Unknown', inplace=True)

df_test['Degree'].fillna('Unknown', inplace=True)
df_train['Degree'].fillna('Unknown', inplace=True)

df_test['Hair'].fillna('Unknown', inplace=True)
df_train['Hair'].fillna('Unknown', inplace=True)

df_test['Gender'].replace('unknown','Unknown',inplace=True)
df_train['Gender'].replace('unknown','Unknown', inplace=True)
df_test['Gender'].replace('f','female', inplace=True)
df_train['Gender'].replace('f','female', inplace=True)

df_test['Country'].replace("""b'Zimbabwe""",'Zimbabwe',inplace=True)

# Apply some preprocessing
# df_test['Experience'] = (df_test["Experience"] - 23).abs()
# df_train['Experience'] = (df_train["Experience"] - 23).abs()

# df_test['Height'] = (df_test['Height'] - 177).abs()
# df_train['Height'] = (df_train['Height'] - 177).abs()

# df_test['Size of City'] = df_test['Size of City'] < 1e7
# df_train['Size of City'] = df_train['Size of City'] < 1e7

# Maybe try sqrt() instead of log
# df_train['Income'] = np.sqrt(df_train['Income'])
# df_train['Income'] = np.log(df_train['Income'])


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_train_and_test = df_train.append(df_test, ignore_index=True, sort=False)

In [6]:
# The following runs target (income) encoding
cols_to_encode = ['House', 'Satisfaction', 'Gender', 'Country', 'Profession', 'Degree', 'Hair']


for col in cols_to_encode:
    unique_values = df_train_and_test[col].unique()
    for unique_value in unique_values:
        matching_rows = df_train['Income'].loc[ df_train[col] == unique_value ]
        value_avg = matching_rows.mean()
        if value_avg > 0:
            df_test[col].replace(unique_value, value_avg, inplace=True)
            df_train[col].replace(unique_value, value_avg, inplace=True)
        else:
            rand_num = random.randint(10000,150000)
            df_test[col].replace(unique_value, rand_num, inplace=True)
            df_train[col].replace(unique_value, rand_num, inplace=True)

In [7]:
Y = df_train['Income']
df_train.drop('Income', inplace=True, axis=1)
X = df_train

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [ ]:
reg = CatBoostRegressor(iterations = 4000).fit(X_train, Y_train)

In [ ]:
predictions = reg.predict(X_val)
score = metrics.mean_absolute_error( Y_val, predictions )
score

In [11]:
reg = CatBoostRegressor(iterations = 6000).fit(X, Y)

0:	learn: 109378.0317339	total: 101ms	remaining: 10m 5s
1:	learn: 106922.5122672	total: 196ms	remaining: 9m 47s
2:	learn: 104544.7584556	total: 300ms	remaining: 9m 59s
3:	learn: 102252.5507849	total: 411ms	remaining: 10m 16s
4:	learn: 100042.6852137	total: 508ms	remaining: 10m 9s
5:	learn: 97913.8284409	total: 610ms	remaining: 10m 9s
6:	learn: 95857.9685625	total: 715ms	remaining: 10m 12s
7:	learn: 93876.6624228	total: 812ms	remaining: 10m 7s
8:	learn: 91943.6660482	total: 918ms	remaining: 10m 10s
9:	learn: 90090.5972667	total: 1.03s	remaining: 10m 15s
10:	learn: 88315.9298357	total: 1.13s	remaining: 10m 14s
11:	learn: 86582.9541978	total: 1.24s	remaining: 10m 18s
12:	learn: 84901.9791521	total: 1.35s	remaining: 10m 21s
13:	learn: 83284.9096873	total: 1.46s	remaining: 10m 22s
14:	learn: 81730.9177101	total: 1.55s	remaining: 10m 18s
15:	learn: 80238.1604403	total: 1.66s	remaining: 10m 20s
16:	learn: 78792.3450750	total: 1.77s	remaining: 10m 21s
17:	learn: 77405.5532015	total: 1.87s	rema

145:	learn: 32494.2623586	total: 16.5s	remaining: 11m 2s
146:	learn: 32431.5836600	total: 16.6s	remaining: 11m 2s
147:	learn: 32365.9232503	total: 16.7s	remaining: 11m 2s
148:	learn: 32310.6809195	total: 16.8s	remaining: 11m 1s
149:	learn: 32209.9586168	total: 17s	remaining: 11m 2s
150:	learn: 32158.6414998	total: 17.1s	remaining: 11m 2s
151:	learn: 32096.2180416	total: 17.2s	remaining: 11m 3s
152:	learn: 32025.1547236	total: 17.3s	remaining: 11m 2s
153:	learn: 31939.4857075	total: 17.5s	remaining: 11m 4s
154:	learn: 31886.5784810	total: 17.7s	remaining: 11m 7s
155:	learn: 31789.8942285	total: 17.8s	remaining: 11m 8s
156:	learn: 31735.4142744	total: 17.9s	remaining: 11m 7s
157:	learn: 31684.5642769	total: 18.1s	remaining: 11m 7s
158:	learn: 31625.7662445	total: 18.2s	remaining: 11m 6s
159:	learn: 31572.5144507	total: 18.3s	remaining: 11m 6s
160:	learn: 31525.9122271	total: 18.4s	remaining: 11m 5s
161:	learn: 31470.9753947	total: 18.5s	remaining: 11m 5s
162:	learn: 31369.1025315	total: 

291:	learn: 26996.7749443	total: 33.1s	remaining: 10m 46s
292:	learn: 26982.7144953	total: 33.2s	remaining: 10m 46s
293:	learn: 26968.5965397	total: 33.3s	remaining: 10m 46s
294:	learn: 26926.1175822	total: 33.4s	remaining: 10m 46s
295:	learn: 26908.9904659	total: 33.5s	remaining: 10m 45s
296:	learn: 26894.5519500	total: 33.6s	remaining: 10m 46s
297:	learn: 26886.4260732	total: 33.7s	remaining: 10m 45s
298:	learn: 26844.8190410	total: 33.9s	remaining: 10m 45s
299:	learn: 26794.8393988	total: 34s	remaining: 10m 45s
300:	learn: 26780.2632317	total: 34.1s	remaining: 10m 45s
301:	learn: 26762.6836183	total: 34.2s	remaining: 10m 46s
302:	learn: 26728.9494378	total: 34.4s	remaining: 10m 45s
303:	learn: 26717.8419757	total: 34.5s	remaining: 10m 45s
304:	learn: 26675.7900044	total: 34.6s	remaining: 10m 45s
305:	learn: 26639.2455237	total: 34.7s	remaining: 10m 45s
306:	learn: 26627.1371441	total: 34.8s	remaining: 10m 45s
307:	learn: 26614.3243395	total: 34.9s	remaining: 10m 45s
308:	learn: 2659

433:	learn: 25006.2729525	total: 48.8s	remaining: 10m 25s
434:	learn: 25004.1876070	total: 48.9s	remaining: 10m 25s
435:	learn: 24998.8410901	total: 49s	remaining: 10m 24s
436:	learn: 24978.0696351	total: 49.1s	remaining: 10m 24s
437:	learn: 24970.5780201	total: 49.2s	remaining: 10m 24s
438:	learn: 24966.4929236	total: 49.3s	remaining: 10m 24s
439:	learn: 24955.8497093	total: 49.4s	remaining: 10m 24s
440:	learn: 24937.9179216	total: 49.5s	remaining: 10m 23s
441:	learn: 24932.7571440	total: 49.6s	remaining: 10m 23s
442:	learn: 24912.2241077	total: 49.7s	remaining: 10m 23s
443:	learn: 24899.5002514	total: 49.8s	remaining: 10m 23s
444:	learn: 24886.0955533	total: 49.9s	remaining: 10m 23s
445:	learn: 24879.4076753	total: 50.1s	remaining: 10m 24s
446:	learn: 24874.9212295	total: 50.3s	remaining: 10m 24s
447:	learn: 24869.4277884	total: 50.4s	remaining: 10m 25s
448:	learn: 24862.7540306	total: 50.6s	remaining: 10m 25s
449:	learn: 24858.4721962	total: 50.7s	remaining: 10m 25s
450:	learn: 2484

577:	learn: 24000.8817582	total: 1m 4s	remaining: 10m 2s
578:	learn: 23993.2137829	total: 1m 4s	remaining: 10m 2s
579:	learn: 23989.9171736	total: 1m 4s	remaining: 10m 2s
580:	learn: 23985.6034468	total: 1m 4s	remaining: 10m 2s
581:	learn: 23982.0835912	total: 1m 4s	remaining: 10m 2s
582:	learn: 23972.0672320	total: 1m 4s	remaining: 10m 2s
583:	learn: 23966.4027908	total: 1m 4s	remaining: 10m 2s
584:	learn: 23964.5425365	total: 1m 5s	remaining: 10m 2s
585:	learn: 23962.2829199	total: 1m 5s	remaining: 10m 1s
586:	learn: 23957.4924281	total: 1m 5s	remaining: 10m 2s
587:	learn: 23952.6188646	total: 1m 5s	remaining: 10m 1s
588:	learn: 23936.6455156	total: 1m 5s	remaining: 10m 2s
589:	learn: 23925.1153287	total: 1m 5s	remaining: 10m 1s
590:	learn: 23922.1218056	total: 1m 5s	remaining: 10m 1s
591:	learn: 23908.9070317	total: 1m 5s	remaining: 10m 1s
592:	learn: 23904.5686723	total: 1m 5s	remaining: 10m 1s
593:	learn: 23902.0123592	total: 1m 6s	remaining: 10m 1s
594:	learn: 23899.1183579	total

...

5375:	learn: 19887.8450791	total: 10m 7s	remaining: 1m 10s
5376:	learn: 19887.6463719	total: 10m 7s	remaining: 1m 10s
5377:	learn: 19887.4621885	total: 10m 7s	remaining: 1m 10s
5378:	learn: 19887.3193853	total: 10m 7s	remaining: 1m 10s
5379:	learn: 19886.9258975	total: 10m 7s	remaining: 1m 10s
5380:	learn: 19886.7963258	total: 10m 7s	remaining: 1m 9s
5381:	learn: 19886.6861336	total: 10m 7s	remaining: 1m 9s
5382:	learn: 19886.6188749	total: 10m 7s	remaining: 1m 9s
5383:	learn: 19886.0801570	total: 10m 7s	remaining: 1m 9s
5384:	learn: 19885.7648685	total: 10m 8s	remaining: 1m 9s
5385:	learn: 19885.5511562	total: 10m 8s	remaining: 1m 9s
5386:	learn: 19885.0334224	total: 10m 8s	remaining: 1m 9s
5387:	learn: 19884.5738409	total: 10m 8s	remaining: 1m 9s
5388:	learn: 19884.4325455	total: 10m 8s	remaining: 1m 8s
5389:	learn: 19884.1745839	total: 10m 8s	remaining: 1m 8s
5390:	learn: 19883.9584913	total: 10m 8s	remaining: 1m 8s
5391:	learn: 19883.6631482	total: 10m 8s	remaining: 1m 8s
5392:	lea

5516:	learn: 19848.5836871	total: 10m 22s	remaining: 54.5s
5517:	learn: 19848.3748546	total: 10m 22s	remaining: 54.4s
5518:	learn: 19848.1539484	total: 10m 23s	remaining: 54.3s
5519:	learn: 19847.9856705	total: 10m 23s	remaining: 54.2s
5520:	learn: 19847.8169268	total: 10m 23s	remaining: 54.1s
5521:	learn: 19847.6224654	total: 10m 23s	remaining: 54s
5522:	learn: 19847.4895266	total: 10m 23s	remaining: 53.8s
5523:	learn: 19847.1900165	total: 10m 23s	remaining: 53.7s
5524:	learn: 19847.1218222	total: 10m 23s	remaining: 53.6s
5525:	learn: 19846.8174346	total: 10m 23s	remaining: 53.5s
5526:	learn: 19846.7139710	total: 10m 23s	remaining: 53.4s
5527:	learn: 19846.4443831	total: 10m 23s	remaining: 53.3s
5528:	learn: 19846.1156304	total: 10m 24s	remaining: 53.2s
5529:	learn: 19845.9923377	total: 10m 24s	remaining: 53s
5530:	learn: 19845.7494513	total: 10m 24s	remaining: 52.9s
5531:	learn: 19845.5682385	total: 10m 24s	remaining: 52.8s
5532:	learn: 19845.4640759	total: 10m 24s	remaining: 52.7s
5

5656:	learn: 19808.7384719	total: 10m 38s	remaining: 38.7s
5657:	learn: 19808.4919165	total: 10m 38s	remaining: 38.6s
5658:	learn: 19808.3522852	total: 10m 38s	remaining: 38.5s
5659:	learn: 19808.1644234	total: 10m 38s	remaining: 38.4s
5660:	learn: 19808.0114000	total: 10m 38s	remaining: 38.3s
5661:	learn: 19807.7749173	total: 10m 39s	remaining: 38.1s
5662:	learn: 19807.4805627	total: 10m 39s	remaining: 38s
5663:	learn: 19806.9782643	total: 10m 39s	remaining: 37.9s
5664:	learn: 19806.1692194	total: 10m 39s	remaining: 37.8s
5665:	learn: 19805.7891441	total: 10m 39s	remaining: 37.7s
5666:	learn: 19805.4693757	total: 10m 39s	remaining: 37.6s
5667:	learn: 19805.1879966	total: 10m 39s	remaining: 37.5s
5668:	learn: 19805.0423401	total: 10m 39s	remaining: 37.4s
5669:	learn: 19804.7677552	total: 10m 39s	remaining: 37.2s
5670:	learn: 19804.5454135	total: 10m 40s	remaining: 37.1s
5671:	learn: 19804.3695469	total: 10m 40s	remaining: 37s
5672:	learn: 19803.9020729	total: 10m 40s	remaining: 36.9s
5

5797:	learn: 19771.5750781	total: 10m 53s	remaining: 22.8s
5798:	learn: 19771.4516285	total: 10m 53s	remaining: 22.7s
5799:	learn: 19771.3127851	total: 10m 54s	remaining: 22.6s
5800:	learn: 19771.1364453	total: 10m 54s	remaining: 22.4s
5801:	learn: 19770.8682061	total: 10m 54s	remaining: 22.3s
5802:	learn: 19770.4488688	total: 10m 54s	remaining: 22.2s
5803:	learn: 19770.0006071	total: 10m 54s	remaining: 22.1s
5804:	learn: 19769.8176231	total: 10m 54s	remaining: 22s
5805:	learn: 19769.5200018	total: 10m 54s	remaining: 21.9s
5806:	learn: 19769.1210248	total: 10m 54s	remaining: 21.8s
5807:	learn: 19768.9734540	total: 10m 54s	remaining: 21.6s
5808:	learn: 19768.6805238	total: 10m 55s	remaining: 21.5s
5809:	learn: 19768.4869333	total: 10m 55s	remaining: 21.4s
5810:	learn: 19768.3561844	total: 10m 55s	remaining: 21.3s
5811:	learn: 19768.1518414	total: 10m 55s	remaining: 21.2s
5812:	learn: 19768.0089985	total: 10m 55s	remaining: 21.1s
5813:	learn: 19767.7183591	total: 10m 55s	remaining: 21s
5

5939:	learn: 19733.9430307	total: 11m 9s	remaining: 6.76s
5940:	learn: 19733.7912680	total: 11m 9s	remaining: 6.65s
5941:	learn: 19733.4428371	total: 11m 9s	remaining: 6.54s
5942:	learn: 19733.1512191	total: 11m 9s	remaining: 6.42s
5943:	learn: 19732.9056040	total: 11m 9s	remaining: 6.31s
5944:	learn: 19732.6501567	total: 11m 10s	remaining: 6.2s
5945:	learn: 19732.4619311	total: 11m 10s	remaining: 6.09s
5946:	learn: 19732.1912714	total: 11m 10s	remaining: 5.97s
5947:	learn: 19732.0172882	total: 11m 10s	remaining: 5.86s
5948:	learn: 19731.7807789	total: 11m 10s	remaining: 5.75s
5949:	learn: 19731.5865024	total: 11m 10s	remaining: 5.63s
5950:	learn: 19731.4065467	total: 11m 10s	remaining: 5.52s
5951:	learn: 19731.2096642	total: 11m 10s	remaining: 5.41s
5952:	learn: 19730.9479692	total: 11m 10s	remaining: 5.3s
5953:	learn: 19730.7743235	total: 11m 11s	remaining: 5.18s
5954:	learn: 19730.4998962	total: 11m 11s	remaining: 5.07s
5955:	learn: 19730.3945752	total: 11m 11s	remaining: 4.96s
5956

In [12]:
predictions = reg.predict(df_test)
df_sub['Total Yearly Income [EUR]'] = predictions
df_sub.to_csv('CatBoost_target_enc_median_6000_iter.csv', index=False)

## Results
Preprocessing, one-hot, unmodified income, linear regression: 695,862  
Preprocessing, target encode, unmodified income, linear regression: 63,226  
Preprocessing, target encode, sqrt income, linear regression: 59296  
Preprocessing (no abs on height and Experience), target encode, sqrt income, linear regression: 59186
Preprocessing, target encode, sqrt income, Catboost: 33219  
Preprocessing, target encode, income, Catboost: 15227  
No Height & Exp processing, target encode, income, Catboost: 15116
No Height & Exp processing, target encode, log income, Catboost: 1500
No processing, target encode, normal income, Catboost: 9648  
No processing, target encode, normal income, Catboost(2000): 9270  
No processing, target encode, normal income, Catboost(3000): 9110